## Name : Diksha Pardeshi
## USC ID : 7961073499
## GitHub username : dikshapardeshi


Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.
uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data
randomly as the training set.

In [1]:
import numpy as np
import pandas as pd
import random

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix, f1_score, hamming_loss, silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


(b) Each instance has three labels: Families, Genus, and Species. Each of the labels
has multiple classes. We wish to solve a multi-class and multi-label problem.
One of the most important approaches to multi-label classi cation is to train a
classi er for each label (binary relevance).

In [3]:
import numpy as np
import pandas as pd

total_rows = len(df)

np.random.seed(total_rows)

train_size = int(total_rows * 0.7)

train_indices = np.random.choice(total_rows, size=train_size, replace=False)

test_indices = [i for i in range(total_rows) if i not in train_indices]

train_data = df.iloc[train_indices].reset_index(drop=True)
test_data = df.iloc[test_indices].reset_index(drop=True)

X_train = train_data.iloc[:, :-4]
X_test = test_data.iloc[:, :-4]

train_family = train_data['Family']
train_genus = train_data['Genus']
train_species = train_data['Species']

test_family = test_data['Family']
test_genus = test_data['Genus']
test_species = test_data['Species']



i. Research exact match and hamming score/ loss methods for evaluating multi-
label classi cation and use them in evaluating the classi ers in this problem.

Exact match: 

The Exact Match Ratio (or Subset Accuracy) evaluates the percentage of instances where every predicted label exactly matches the corresponding true labels. For each instance, the set of predicted labels must be identical to the true labels, irrespective of the order. This is a stringent metric, demanding an exact match between the predicted and actual labels for each instance.

Hamming Loss : 

The Hamming Loss quantifies the fraction of incorrect label predictions across all instances and labels. It computes the average number of incorrect labels predicted for each instance. A lower Hamming Loss signifies better performance, as it reflects fewer errors in label predictions throughout the dataset.


ii. Train a SVM for each of the labels

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report


In [5]:
def metrics(y_test, y_pred):
    error = np.sum(np.not_equal(y_test.values, y_pred.values))
    hamming = error / (y_test.size)
    
    exact_ratio = np.mean(np.all(y_test.values == y_pred.values, axis=1))
    
    print(f"Hamming Loss: {hamming:.4f}")
    print(f"Exact Match Ratio: {exact_ratio:.4f}")
    
    return hamming, exact_ratio


In [6]:
def eval_classifiers(x_test, y_test, classifiers):
    y_pred = pd.DataFrame(columns=y_test.columns)
    for label in y_test.columns:
        model = classifiers[label]
        y_pred[label] = model.predict(x_test)
    return metrics(y_test, y_pred)


In [7]:
def find_best_model(classifier, kwargs, x_train, y_train, x_test, y_test):
    model = GridSearchCV(estimator=classifier, **kwargs, n_jobs=-1)
    model.fit(x_train, y_train)
    
    results = {key: [] for key in model.cv_results_['params'][0].keys()}
    for param in model.cv_results_['params']:
        for key, value in param.items():
            results[key].append(value)
    
    means = np.round(np.array(model.cv_results_['mean_test_score']), 5)
    results['mean_test_score'] = means
    result_data = pd.DataFrame(results)
    display(result_data)


    best_params = {key: float(value) for key, value in model.best_params_.items()}
    print("\nBest Parameters:", best_params)
    print(classification_report(y_test, model.predict(x_test)))
    
    return model


In [8]:
kwargs = {
    'scoring': 'f1_weighted',
    'param_grid': {
        'C': np.logspace(1, 4, 4), 
        'gamma': np.logspace(-3, 6, 10)
    },
    'cv': StratifiedKFold(10, random_state=69, shuffle=True)
}
summary={}


In [9]:
gaussian_SVC_without_std = {}
method = "Gaussian SVC without Standardization"
print(f"Method: {method}")
print("Label: Family")

gaussian_SVC_without_std['Family'] = find_best_model(SVC(kernel='rbf'), kwargs, X_train, train_family, X_test, test_family)


Method: Gaussian SVC without Standardization
Label: Family


,C,gamma,mean_test_score
0,10.0,0.001,0.84366
1,10.0,0.010,0.92498
2,10.0,0.100,0.96434
3,10.0,1.000,0.99140
4,10.0,10.000,0.98686
5,10.0,100.000,0.78345
6,10.0,1000.000,0.49165
7,10.0,10000.000,0.47291
8,10.0,100000.000,0.46983
9,10.0,1000000.000,0.46983



Best Parameters: {'C': 100.0, 'gamma': 1.0}
                 precision    recall  f1-score   support

      Bufonidae       0.95      0.90      0.92        20
  Dendrobatidae       0.99      1.00      0.99       185
        Hylidae       0.99      0.98      0.99       637
Leptodactylidae       0.99      0.99      0.99      1317

       accuracy                           0.99      2159
      macro avg       0.98      0.97      0.97      2159
   weighted avg       0.99      0.99      0.99      2159



In [10]:
method = "Gaussian SVC without Standardization"
print(f"Method: {method}")
print("Label: Genus")


gaussian_SVC_without_std['Genus'] = find_best_model(SVC(kernel='rbf'), kwargs, X_train, train_genus, X_test, test_genus)


Method: Gaussian SVC without Standardization
Label: Genus


,C,gamma,mean_test_score
0,10.0,0.001,0.71497
1,10.0,0.010,0.91699
2,10.0,0.100,0.97080
3,10.0,1.000,0.99109
4,10.0,10.000,0.97952
5,10.0,100.000,0.72175
6,10.0,1000.000,0.44298
7,10.0,10000.000,0.42522
8,10.0,100000.000,0.42276
9,10.0,1000000.000,0.42276



Best Parameters: {'C': 10.0, 'gamma': 1.0}
               precision    recall  f1-score   support

    Adenomera       0.99      1.00      0.99      1242
     Ameerega       0.99      1.00      1.00       185
Dendropsophus       0.93      0.93      0.93        76
    Hypsiboas       0.99      0.99      0.99       487
Leptodactylus       0.96      1.00      0.98        75
Osteocephalus       0.93      0.84      0.89        32
     Rhinella       1.00      0.85      0.92        20
       Scinax       0.93      0.98      0.95        42

     accuracy                           0.99      2159
    macro avg       0.97      0.95      0.96      2159
 weighted avg       0.99      0.99      0.99      2159



In [11]:
method = "Gaussian SVC without Standardization"
print(f"Method: {method}")
print("Label: Species")

gaussian_SVC_without_std['Species'] = find_best_model(SVC(kernel='rbf'), kwargs, X_train, train_species, X_test, test_species)


Method: Gaussian SVC without Standardization
Label: Species


,C,gamma,mean_test_score
0,10.0,0.001,0.78884
1,10.0,0.010,0.93319
2,10.0,0.100,0.97528
3,10.0,1.000,0.99042
4,10.0,10.000,0.97593
5,10.0,100.000,0.64457
6,10.0,1000.000,0.33145
7,10.0,10000.000,0.31735
8,10.0,100000.000,0.31497
9,10.0,1000000.000,0.31497



Best Parameters: {'C': 10.0, 'gamma': 1.0}
                        precision    recall  f1-score   support

        AdenomeraAndre       0.97      0.98      0.98       198
AdenomeraHylaedactylus       1.00      1.00      1.00      1044
    Ameeregatrivittata       0.99      1.00      1.00       185
            HylaMinuta       0.93      0.93      0.93        76
  HypsiboasCinerascens       0.98      0.98      0.98       132
     HypsiboasCordobae       0.99      0.99      0.99       355
   LeptodactylusFuscus       0.96      1.00      0.98        75
 OsteocephalusOophagus       0.93      0.84      0.89        32
     Rhinellagranulosa       1.00      0.85      0.92        20
           ScinaxRuber       0.93      0.98      0.95        42

              accuracy                           0.99      2159
             macro avg       0.97      0.96      0.96      2159
          weighted avg       0.99      0.99      0.99      2159



In [12]:
print(gaussian_SVC_without_std)

{'Family': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([   10.,   100.,  1000., 10000.]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04,
       1.e+05, 1.e+06])},
             scoring='f1_weighted'), 'Genus': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([   10.,   100.,  1000., 10000.]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04,
       1.e+05, 1.e+06])},
             scoring='f1_weighted'), 'Species': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([   10.,   100.,  1000., 10000.]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1

In [13]:
print(f"Summary of Evaluation: {method}")
summary[method] = eval_classifiers(X_test, test_data.iloc[:, -4:-1], gaussian_SVC_without_std)


Summary of Evaluation: Gaussian SVC without Standardization
Hamming Loss: 0.0108
Exact Match Ratio: 0.9856


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [15]:

gaussian_SVC_with_std = {}
method = "Gaussian SVC with Standardization"
print(f"Method: {method}")
print("Label: Family")

gaussian_SVC_with_std['Family'] = find_best_model(SVC(kernel='rbf'), kwargs, X_train_scaled, train_family, X_test_scaled, test_family)


Method: Gaussian SVC with Standardization
Label: Family


,C,gamma,mean_test_score
0,10.0,0.001,0.94564
1,10.0,0.010,0.98728
2,10.0,0.100,0.99060
3,10.0,1.000,0.90869
4,10.0,10.000,0.56499
5,10.0,100.000,0.47320
6,10.0,1000.000,0.46983
7,10.0,10000.000,0.46983
8,10.0,100000.000,0.46983
9,10.0,1000000.000,0.46983



Best Parameters: {'C': 10.0, 'gamma': 0.1}
                 precision    recall  f1-score   support

      Bufonidae       1.00      0.85      0.92        20
  Dendrobatidae       0.99      1.00      1.00       185
        Hylidae       0.98      0.99      0.99       637
Leptodactylidae       0.99      0.99      0.99      1317

       accuracy                           0.99      2159
      macro avg       0.99      0.96      0.97      2159
   weighted avg       0.99      0.99      0.99      2159



In [16]:
method = "Gaussian SVC with Standardization"
print(f"Method: {method}")
print("Label: Genus")

gaussian_SVC_with_std['Genus'] = find_best_model(SVC(kernel='rbf'), kwargs, X_train, train_genus, X_test, test_genus)


Method: Gaussian SVC with Standardization
Label: Genus


,C,gamma,mean_test_score
0,10.0,0.001,0.71497
1,10.0,0.010,0.91699
2,10.0,0.100,0.97080
3,10.0,1.000,0.99109
4,10.0,10.000,0.97952
5,10.0,100.000,0.72175
6,10.0,1000.000,0.44298
7,10.0,10000.000,0.42522
8,10.0,100000.000,0.42276
9,10.0,1000000.000,0.42276



Best Parameters: {'C': 10.0, 'gamma': 1.0}
               precision    recall  f1-score   support

    Adenomera       0.99      1.00      0.99      1242
     Ameerega       0.99      1.00      1.00       185
Dendropsophus       0.93      0.93      0.93        76
    Hypsiboas       0.99      0.99      0.99       487
Leptodactylus       0.96      1.00      0.98        75
Osteocephalus       0.93      0.84      0.89        32
     Rhinella       1.00      0.85      0.92        20
       Scinax       0.93      0.98      0.95        42

     accuracy                           0.99      2159
    macro avg       0.97      0.95      0.96      2159
 weighted avg       0.99      0.99      0.99      2159



In [17]:
method = "Gaussian SVC with Standardization"
print(f"Method: {method}")
print("Label: Species")

gaussian_SVC_with_std['Species'] = find_best_model(SVC(kernel='rbf'), kwargs, X_train_scaled, train_species, X_test_scaled, test_species)


Method: Gaussian SVC with Standardization
Label: Species


,C,gamma,mean_test_score
0,10.0,0.001,0.96602
1,10.0,0.010,0.98935
2,10.0,0.100,0.98454
3,10.0,1.000,0.83140
4,10.0,10.000,0.38116
5,10.0,100.000,0.31810
6,10.0,1000.000,0.31497
7,10.0,10000.000,0.31497
8,10.0,100000.000,0.31497
9,10.0,1000000.000,0.31497



Best Parameters: {'C': 1000.0, 'gamma': 0.01}
                        precision    recall  f1-score   support

        AdenomeraAndre       0.99      0.98      0.99       198
AdenomeraHylaedactylus       1.00      1.00      1.00      1044
    Ameeregatrivittata       0.99      0.99      0.99       185
            HylaMinuta       0.90      0.95      0.92        76
  HypsiboasCinerascens       0.97      0.98      0.97       132
     HypsiboasCordobae       0.99      0.99      0.99       355
   LeptodactylusFuscus       0.97      0.99      0.98        75
 OsteocephalusOophagus       0.96      0.81      0.88        32
     Rhinellagranulosa       0.95      0.95      0.95        20
           ScinaxRuber       0.93      0.98      0.95        42

              accuracy                           0.99      2159
             macro avg       0.97      0.96      0.96      2159
          weighted avg       0.99      0.99      0.99      2159



In [21]:
gaussian_SVC_with_std

{'Family': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
              estimator=SVC(), n_jobs=-1,
              param_grid={'C': array([   10.,   100.,  1000., 10000.]),
                          'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04,
        1.e+05, 1.e+06])},
              scoring='f1_weighted'),
 'Genus': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
              estimator=SVC(), n_jobs=-1,
              param_grid={'C': array([   10.,   100.,  1000., 10000.]),
                          'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04,
        1.e+05, 1.e+06])},
              scoring='f1_weighted'),
 'Species': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
              estimator=SVC(), n_jobs=-1,
              param_grid={'C': array([   10.,   100.,  1000., 10000.]),
                          'gamma': array([1.e-03, 1.

In [23]:
print(f"Summary of Evaluation: {method}")
summary[method] = eval_classifiers(X_test, test_data.iloc[:, -4:-1], gaussian_SVC_with_std)


Summary of Evaluation: Gaussian SVC with Standardization
Hamming Loss: 0.2145
Exact Match Ratio: 0.5910


In [24]:
SVM_L1_penalized_classifiers = {}

kwargs = {
    'scoring': 'f1_weighted',
    'param_grid': {
        'C': np.logspace(1, 5, 10),
    },
    'cv': StratifiedKFold(10, random_state=69, shuffle=True)
}




In [27]:
method = "Support Vector Classifier with L1 Penalty"
print(f"Method: {method}")
print("Label: Family")

SVM_L1_penalized_classifiers['Family'] = find_best_model(LinearSVC(penalty='l1',dual= False, max_iter=10000), kwargs, X_train_scaled, train_family, X_test_scaled, test_family)


Method: Support Vector Classifier with L1 Penalty
Label: Family


,C,mean_test_score
0,10.000000,0.92888
1,27.825594,0.92908
2,77.426368,0.92908
3,215.443469,0.92908
4,599.484250,0.92908
5,1668.100537,0.92908
6,4641.588834,0.92908
7,12915.496650,0.92908
8,35938.136638,0.92908
9,100000.000000,0.92908



Best Parameters: {'C': 27.825594022071243}
                 precision    recall  f1-score   support

      Bufonidae       0.00      0.00      0.00        20
  Dendrobatidae       0.92      0.87      0.89       185
        Hylidae       0.90      0.92      0.91       637
Leptodactylidae       0.96      0.97      0.96      1317

       accuracy                           0.94      2159
      macro avg       0.70      0.69      0.69      2159
   weighted avg       0.93      0.94      0.93      2159



In [28]:
method = "Support Vector Classifier with L1 Penalty"
print(f"Method: {method}")
print("Label: Genus")

SVM_L1_penalized_classifiers['Genus'] = find_best_model(LinearSVC(penalty='l1',dual= False, max_iter=10000), kwargs, X_train_scaled, train_genus, X_test_scaled, test_genus)


Method: Support Vector Classifier with L1 Penalty
Label: Genus


/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.p

,C,mean_test_score
0,10.000000,0.94554
1,27.825594,0.94577
2,77.426368,0.94577
3,215.443469,0.94577
4,599.484250,0.94577
5,1668.100537,0.94577
6,4641.588834,0.94577
7,12915.496650,0.94577
8,35938.136638,0.94577
9,100000.000000,0.94577



Best Parameters: {'C': 27.825594022071243}
               precision    recall  f1-score   support

    Adenomera       0.96      0.99      0.98      1242
     Ameerega       0.93      0.92      0.93       185
Dendropsophus       0.83      0.63      0.72        76
    Hypsiboas       0.94      0.97      0.95       487
Leptodactylus       0.93      0.89      0.91        75
Osteocephalus       0.95      0.59      0.73        32
     Rhinella       0.90      0.45      0.60        20
       Scinax       0.93      0.93      0.93        42

     accuracy                           0.95      2159
    macro avg       0.92      0.80      0.84      2159
 weighted avg       0.95      0.95      0.95      2159



In [29]:
method = "Support Vector Classifier with L1 Penalty"
print(f"Method: {method}")
print("Label: Species")

SVM_L1_penalized_classifiers['Species'] = find_best_model(LinearSVC(penalty='l1',dual= False, max_iter=10000), kwargs, X_train_scaled, train_species, X_test_scaled, test_species)


Method: Support Vector Classifier with L1 Penalty
Label: Species


,C,mean_test_score
0,10.000000,0.95262
1,27.825594,0.95262
2,77.426368,0.95262
3,215.443469,0.95262
4,599.484250,0.95262
5,1668.100537,0.95262
6,4641.588834,0.95262
7,12915.496650,0.95262
8,35938.136638,0.95262
9,100000.000000,0.95262



Best Parameters: {'C': 10.0}
                        precision    recall  f1-score   support

        AdenomeraAndre       0.92      0.94      0.93       198
AdenomeraHylaedactylus       0.99      1.00      0.99      1044
    Ameeregatrivittata       0.93      0.92      0.93       185
            HylaMinuta       0.83      0.71      0.77        76
  HypsiboasCinerascens       0.92      0.95      0.94       132
     HypsiboasCordobae       0.95      0.97      0.96       355
   LeptodactylusFuscus       0.96      0.91      0.93        75
 OsteocephalusOophagus       0.95      0.62      0.75        32
     Rhinellagranulosa       0.89      0.85      0.87        20
           ScinaxRuber       0.91      1.00      0.95        42

              accuracy                           0.96      2159
             macro avg       0.93      0.89      0.90      2159
          weighted avg       0.96      0.96      0.96      2159



In [30]:
print("Evaluation Summary: ", method)
summary[method] = eval_classifiers(X_test_scaled, test_data[['Family', 'Genus', 'Species']],SVM_L1_penalized_classifiers)


Evaluation Summary:  Support Vector Classifier with L1 Penalty
Hamming Loss: 0.0509
Exact Match Ratio: 0.9171


Repeat 1(b)iii by using SMOTE or any other method you know to remedy
class imbalance. Report your conclusions about the classi ers you trained.

In [38]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
SVM_smote_classifiers = {}

def smote_param_search(classifier, settings, x_train, y_train, x_test, y_test):
    naive_model = Pipeline([
        ('sampling', SMOTE()),  
        ('classification', classifier)  
    ])
    selected_model = find_best_model(naive_model, settings, x_train, y_train, x_test, y_test)
    return selected_model


In [39]:
kwargs = {
    'scoring': 'f1_weighted',  
    'param_grid': {
        'classification__C': np.logspace(1, 5, 10)  
    },
    'cv': StratifiedKFold(10, random_state=69, shuffle=True)  
}




In [43]:
method = "SVM with L1 penalty and SMOTE"
print("Method: ", method)
print("Label: Family")

SVM_smote_classifiers['Family'] = smote_param_search(
    LinearSVC(penalty='l1', dual=False, max_iter=10000),
    kwargs, X_train_scaled, train_family, X_test_scaled, test_family
)


Method:  SVM with L1 penalty and SMOTE
Label: Family


/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/dsci552/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,classification__C,mean_test_score
0,10.000000,0.92142
1,27.825594,0.92291
2,77.426368,0.92167
3,215.443469,0.92242
4,599.484250,0.92139
5,1668.100537,0.92101
6,4641.588834,0.92362
7,12915.496650,0.92275
8,35938.136638,0.92188
9,100000.000000,0.92029



Best Parameters: {'classification__C': 4641.588833612777}
                 precision    recall  f1-score   support

      Bufonidae       0.33      0.90      0.48        20
  Dendrobatidae       0.83      0.97      0.90       185
        Hylidae       0.94      0.89      0.92       637
Leptodactylidae       0.97      0.94      0.96      1317

       accuracy                           0.93      2159
      macro avg       0.77      0.93      0.81      2159
   weighted avg       0.94      0.93      0.93      2159



In [50]:
method = "SVM with L1 penalty and SMOTE"
print("Method: ", method)
print("Label: Genus")

SVM_smote_classifiers['Genus'] = smote_param_search(
    LinearSVC(penalty='l1', dual=False, max_iter=40000),
    kwargs, X_train_scaled, train_genus, X_test_scaled, test_genus)


Method:  SVM with L1 penalty and SMOTE
Label: Genus


,classification__C,mean_test_score
0,10.000000,0.91824
1,27.825594,0.91860
2,77.426368,0.91857
3,215.443469,0.91761
4,599.484250,0.91704
5,1668.100537,0.91662
6,4641.588834,0.91807
7,12915.496650,0.92011
8,35938.136638,0.91735
9,100000.000000,0.91857



Best Parameters: {'classification__C': 12915.496650148827}
               precision    recall  f1-score   support

    Adenomera       0.98      0.91      0.95      1242
     Ameerega       0.87      0.94      0.90       185
Dendropsophus       0.53      0.83      0.64        76
    Hypsiboas       0.97      0.93      0.95       487
Leptodactylus       0.92      0.92      0.92        75
Osteocephalus       0.51      0.88      0.64        32
     Rhinella       0.43      0.95      0.59        20
       Scinax       0.82      0.98      0.89        42

     accuracy                           0.92      2159
    macro avg       0.75      0.92      0.81      2159
 weighted avg       0.94      0.92      0.92      2159



In [56]:
method = "SVM with L1 penalty and SMOTE"
print("Method: ", method)
print("Label: Species")

SVM_smote_classifiers['Species'] = smote_param_search(
    LinearSVC(penalty='l1', dual=False, max_iter=100000),
    kwargs, X_train_scaled, train_species, X_test_scaled, test_species)


Method:  SVM with L1 penalty and SMOTE
Label: Species


,classification__C,mean_test_score
0,10.000000,0.95383
1,27.825594,0.95306
2,77.426368,0.95220
3,215.443469,0.95290
4,599.484250,0.95322
5,1668.100537,0.95383
6,4641.588834,0.95427
7,12915.496650,0.95343
8,35938.136638,0.95352
9,100000.000000,0.95413



Best Parameters: {'classification__C': 4641.588833612777}
                        precision    recall  f1-score   support

        AdenomeraAndre       0.97      0.93      0.95       198
AdenomeraHylaedactylus       0.99      0.99      0.99      1044
    Ameeregatrivittata       0.93      0.91      0.92       185
            HylaMinuta       0.78      0.84      0.81        76
  HypsiboasCinerascens       0.93      0.94      0.93       132
     HypsiboasCordobae       0.97      0.94      0.95       355
   LeptodactylusFuscus       0.92      0.92      0.92        75
 OsteocephalusOophagus       0.88      0.88      0.88        32
     Rhinellagranulosa       0.69      1.00      0.82        20
           ScinaxRuber       0.78      0.95      0.86        42

              accuracy                           0.96      2159
             macro avg       0.88      0.93      0.90      2159
          weighted avg       0.96      0.96      0.96      2159



In [57]:
SVM_smote_classifiers

{'Family': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
              estimator=Pipeline(steps=[('sampling', SMOTE()),
                                        ('classification',
                                         LinearSVC(dual=False, max_iter=10000,
                                                   penalty='l1'))]),
              n_jobs=-1,
              param_grid={'classification__C': array([1.00000000e+01, 2.78255940e+01, 7.74263683e+01, 2.15443469e+02,
        5.99484250e+02, 1.66810054e+03, 4.64158883e+03, 1.29154967e+04,
        3.59381366e+04, 1.00000000e+05])},
              scoring='f1_weighted'),
 'Genus': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=69, shuffle=True),
              estimator=Pipeline(steps=[('sampling', SMOTE()),
                                        ('classification',
                                         LinearSVC(dual=False, max_iter=40000,
                                                   penalty

In [58]:
print("Evaluation Summary: ", method)
summary[method] = eval_classifiers(X_test_scaled, test_data[['Family', 'Genus', 'Species']],SVM_smote_classifiers)


Evaluation Summary:  SVM with L1 penalty and SMOTE
Hamming Loss: 0.0653
Exact Match Ratio: 0.8740


In [59]:
final_summary = pd.DataFrame(data=summary)
final_summary.index = ['Hamming Loss', 'Exact Match Ratio']
final_summary


,SVM with L1 penalty and SMOTE
Hamming Loss,0.065308
Exact Match Ratio,0.874016




**Findings**

The Gaussian Kernel SVM has the highest Average Exact Match Loss.

For the L1-Penalized SVM, both Average Exact Match Loss and Average Hamming Loss are higher compared to the Gaussian Kernel SVM.

After using SMOTE to balance the classes, the Average Exact Match Loss for the L1-Penalized SVM decreases.

**Conclusion:**  

Based on Average Exact Match Loss and Average Hamming Loss, the Gaussian Kernel SVM performs the best across all models.

(2) K-Means Clustering on a Multi-Class and Multi-Label Data Set

In [60]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline

In [61]:
def find_optimal_k(max_clusters, x_train_data, seed):
    optimal_k, highest_score = 2, 0
    for clusters in range(2, max_clusters + 1):
        kmeans_model = KMeans(n_clusters=clusters, random_state=seed)
        labels = kmeans_model.fit_predict(x_train_data)
        silhouette_avg = silhouette_score(x_train_data, labels)
        if silhouette_avg > highest_score:
            optimal_k = clusters
            highest_score = silhouette_avg
    print(f"Optimal K: {optimal_k}")
    return optimal_k


In [62]:
def assign_majority_class_labels(optimal_k, cluster_labels, labels_train):
    majority_labels = pd.DataFrame(columns=labels_train.columns)
    for cluster in range(optimal_k):
        cluster_indices, = np.where(cluster_labels == cluster)
        cluster_samples = labels_train.iloc[cluster_indices, :]
        majority_row = []
        for column in labels_train.columns:
            majority_class = cluster_samples[column].value_counts().idxmax()
            majority_row.append(majority_class)
        majority_labels.loc[cluster] = majority_row
    return majority_labels



In [63]:
def calculate_hamming_metrics(majority_labels, cluster_labels, labels_train):
    misclassified_labels = 0
    for cluster in range(len(majority_labels)):
        cluster_indices, = np.where(cluster_labels == cluster)
        for actual_labels in labels_train.iloc[cluster_indices].values:
            misclassification = (actual_labels != majority_labels.iloc[cluster].values)
            misclassified_labels += np.sum(misclassification)
    hamming_distance = misclassified_labels / labels_train.shape[0]
    hamming_loss = misclassified_labels / (labels_train.shape[0] * labels_train.shape[1])
    return hamming_distance, hamming_loss


In [67]:
##chatgpt prompt : give the structure of a basic monte carlo with multiple iterations

def monte_carlo(iterations, x_train_data, y_train_data):
    all_hamming_distances, all_hamming_losses = [], []
    for i in range(iterations):
        print(f"Running iteration {i+1}")
        optimal_k = find_optimal_k(iterations, x_train_data, i)
        kmeans_model = KMeans(n_clusters=optimal_k, random_state=i)
        cluster_labels = kmeans_model.fit_predict(x_train_data)
        majority_class_labels = assign_majority_class_labels(optimal_k, cluster_labels, y_train_data)
        display(majority_class_labels)
        hamming_dist, hamming_loss = calculate_hamming_metrics(majority_class_labels, cluster_labels, y_train_data)
        all_hamming_distances.append(hamming_dist)
        all_hamming_losses.append(hamming_loss)
        print(f"Hamming Distance: {round(hamming_dist, 4)}")
        print(f"Hamming Loss: {round(hamming_loss, 4)}\n")
    return all_hamming_distances, all_hamming_losses


In [68]:
def report_results(hamming_distances):
    avg_hamming_distance = np.mean(hamming_distances)
    std_hamming_distance = np.std(hamming_distances)
    print(f"Average Hamming Distance: {avg_hamming_distance:.4f}")
    print(f"Standard Deviation of Hamming Distance: {std_hamming_distance:.4f}")


In [69]:
hamming_distances, hamming_losses = monte_carlo(50, df.iloc[:, :-4], df.iloc[:, -4:-1])
report_results(hamming_distances)


Running iteration 1
Optimal K: 4


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Hylidae,Hypsiboas,HypsiboasCinerascens
3,Leptodactylidae,Adenomera,AdenomeraAndre


Hamming Distance: 0.7022
Hamming Loss: 0.2341

Running iteration 2
Optimal K: 4


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Leptodactylidae,Adenomera,AdenomeraAndre
3,Hylidae,Hypsiboas,HypsiboasCordobae


Hamming Distance: 0.7358
Hamming Loss: 0.2453

Running iteration 3
Optimal K: 4


...

Hamming Distance: 0.6673
Hamming Loss: 0.2224

Running iteration 49
Optimal K: 3


,Family,Genus,Species
0,Dendrobatidae,Ameerega,Ameeregatrivittata
1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2,Hylidae,Hypsiboas,HypsiboasCordobae


Hamming Distance: 0.7155
Hamming Loss: 0.2385

Running iteration 50
Optimal K: 3


,Family,Genus,Species
0,Hylidae,Hypsiboas,HypsiboasCordobae
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Leptodactylidae,Adenomera,AdenomeraHylaedactylus


Hamming Distance: 0.8905
Hamming Loss: 0.2968

Average Hamming Distance: 0.7187
Standard Deviation of Hamming Distance: 0.1296


(c)Now for each cluster you have a majority label triplet (family, genus, species).
Calculate the average Hamming distance, Hamming score, and Hamming loss5
between the true labels and the labels assigned by clusters.

In [73]:
print("Mean Hamming Distance:", round(np.mean(hamming_distances), 5))
print("Mean Hamming Loss:", round(np.mean(hamming_losses), 5))
print("Mean Hamming Score:", round(1 - np.mean(hamming_losses), 5))


Mean Hamming Distance: 0.71867
Mean Hamming Loss: 0.23956
Mean Hamming Score: 0.76044


In [72]:
print(type(hamming_loss))


<class 'function'>


3. ISLR 12.6.2

![image1](../image1.jpeg)

![image2](../image2.jpeg)